## Import Libraries

In [1]:
import os
import warnings
import joblib
import numpy as np
import pandas as pd
import panel as pn
import holoviews as hv
import hvplot.pandas

print("All imports successful")

# Initialize Panel and HoloViews with Bokeh
print("Starting to initialize extensions...")
hv.extension('bokeh')
print("HoloViews extension loaded")
pn.extension()
print("Panel extension loaded")
warnings.filterwarnings('ignore')
print("Warnings filtered")


class ToxinPredictionDashboard:
    """
    A Panel-based dashboard for predicting toxin levels based on user-input parameters.
    """

    def __init__(self):
        """Initialize the dashboard by loading models and setting up widgets."""
        print("Starting dashboard initialization...")
        
        # Attempt to load trained model pipelines
        print("Attempting to load models...")
        self.models_loaded = self._load_models()
        print(f"Models loaded: {self.models_loaded}")

        # Define parameter ranges for the 9 features
        print("Setting up feature ranges...")
        self.feature_ranges = {
            'Q':          {'min': -333, 'max': 283, 'initial': 20},
            'Q_1m':       {'min': -300, 'max': 1020, 'initial': 90},
            'WaterTemp':  {'min': 13,   'max': 26.6,   'initial': 22.63},
            'DO':         {'min': 6.6,    'max': 10.7,   'initial': 8.17},
            'pH':         {'min': 7.0,  'max': 8.8,  'initial': 7.96},
            'SpecCond.':  {'min': 56,  'max': 10834, 'initial': 2910},
            'OrgC':       {'min': 1.4,  'max': 5.5,  'initial': 2.4},
            'OrgN':       {'min': 0.0,  'max': 1.6,  'initial': 0.3},
            'PO4':        {'min': 0.0,  'max': 0.53,  'initial': 0.08}
        }
        print("Feature ranges set")

        print("Setting up feature descriptions...")
        # Descriptions for display
        self.feature_descriptions = {
            'Q':          'Flow Rate (m³/s)',
            'Q_1m':       'Flow Rate 1 Month Ago (m³/s)',
            'WaterTemp':  'Water Temperature (°C)',
            'DO':         'Dissolved Oxygen (mg/L)',
            'pH':         'pH',
            'SpecCond.':  'Specific Conductance (μS/cm)',
            'OrgC':       'Organic Carbon (mg/L)',
            'OrgN':       'Organic Nitrogen (mg/L)',
            'PO4':        'Phosphate (mg/L)'
        }
        print("Feature descriptions set")

        print("Creating feature sliders...")
        # Create sliders for user input
        self.feature_sliders = self._create_feature_sliders()
        print("Feature sliders created")

        print("Setting up sensitivity analysis selector...")
        # Create dropdown for sensitivity analysis
        self.sensitivity_feature_selector = pn.widgets.Select(
            name='Select Feature for Sensitivity Analysis',
            options=list(self.feature_descriptions.values()),
            value=list(self.feature_descriptions.values())[0],
            width=400
        )
        self.sensitivity_feature_selector.param.watch(
            self.update_sensitivity_analysis_plot, 'value'
        )
        print("Sensitivity analysis selector created")

        print("Creating output panes...")
        # Create output panes for results
        self.random_forest_markdown = pn.pane.Markdown("")
        self.xgboost_markdown = pn.pane.Markdown("")
        self.model_comparison_plot = pn.pane.HoloViews()
        self.sensitivity_analysis_plot = pn.pane.HoloViews()
        print("Output panes created")

        print("Triggering initial prediction...")
        # Trigger an initial prediction
        self.update_predictions(None)
        print("Initial prediction completed")

    def _load_models(self) -> bool:
        """
        Load the Random Forest and XGBoost model pipelines.
        Returns True if both models load successfully, otherwise False.
        """
        try:
            print("Starting to load Random Forest model...")
            self.random_forest_model = joblib.load(r'.\Model\rf_pipeline.joblib')
            print("Random Forest model loaded successfully")
            
            print("Starting to load XGBoost model...")
            self.xgboost_model = joblib.load(r'.\Model\xgb_pipeline.joblib')
            print("XGBoost model loaded successfully")
            return True
        except Exception as e:
            print(f"Error loading models: {str(e)}")
            import traceback
            print(traceback.format_exc())
            return False

    def _create_feature_sliders(self):
        """
        Create Panel FloatSlider widgets for each feature based on the defined ranges.
        """
        print("Creating sliders...")
        sliders = {}
        for feature, range_info in self.feature_ranges.items():
            print(f"Creating slider for {feature}")
            slider = pn.widgets.FloatSlider(
                name=self.feature_descriptions[feature],
                start=range_info['min'],
                end=range_info['max'],
                value=range_info['initial'],
                step=(range_info['max'] - range_info['min']) / 100,
                width=400
            )
            slider.param.watch(self.update_predictions, 'value')
            sliders[feature] = slider
        print("All sliders created")
        return sliders

    def get_feature_key(self, description: str) -> str:
        """
        Convert a user-facing description back to the internal feature name.
        """
        reversed_map = {desc: feat for feat, desc in self.feature_descriptions.items()}
        return reversed_map[description]

    def update_predictions(self, event):
        """
        Update both the Random Forest and XGBoost predictions and their probabilities,
        then regenerate the comparison plot.
        """
        print("Updating predictions...")
        # Gather current slider values
        current_values = {f: s.value for f, s in self.feature_sliders.items()}
        feature_order = ["Q", "Q_1m", "WaterTemp", "DO", "pH", "SpecCond.", "OrgC", "OrgN", "PO4"]
        sample_df = pd.DataFrame([current_values])[feature_order]

        # Random Forest predictions
        rf_label = self.random_forest_model.predict(sample_df)[0]  # 0 or 1
        rf_prob = self.random_forest_model.predict_proba(sample_df)[0][1]

        # XGBoost predictions
        xgb_label = self.xgboost_model.predict(sample_df)[0]  # 0 or 1
        xgb_prob = self.xgboost_model.predict_proba(sample_df)[0][1]

        print("Updating markdown displays...")
        # Update the markdown displays
        rf_result_text = "High (>4000)" if rf_label == 1 else "Low (≤4000)"
        xgb_result_text = "High (>4000)" if xgb_label == 1 else "Low (≤4000)"

        self.random_forest_markdown.object = (
            f"### Random Forest Prediction\n"
            f"**Result:** {rf_result_text} toxin production\n"
            f"**Probability:** {rf_prob:.2%} chance of high toxin production"
        )
        self.xgboost_markdown.object = (
            f"### XGBoost Prediction\n"
            f"**Result:** {xgb_result_text} toxin production\n"
            f"**Probability:** {xgb_prob:.2%} chance of high toxin production"
        )

        print("Creating comparison plot...")
        # Create bar plot comparing probabilities
        comparison_df = pd.DataFrame({
            'Model': ['Random Forest', 'XGBoost'],
            'Probability': [rf_prob, xgb_prob]
        })

        comparison_plot = comparison_df.hvplot.bar(
            x='Model',
            y='Probability',
            title='Model Probability Comparison',
            width=400,
            height=300,
            color=['#1f77b4', '#ff7f0e']
        ).opts(
            tools=['hover'],
            yformatter='%.2f',
            toolbar=None,
            fontsize={'title': 12, 'labels': 10, 'xticks': 10, 'yticks': 10}
        )
        self.model_comparison_plot.object = comparison_plot

        print("Updating sensitivity plot...")
        # Automatically update the sensitivity plot (optional)
        self.update_sensitivity_analysis_plot(None)
        print("Predictions updated successfully")

    def update_sensitivity_analysis_plot(self, event):
        """
        Generate a line plot showing how the prediction probability changes
        for a single feature, while holding other features constant.
        """
        print("Updating sensitivity analysis...")
        current_values = {f: s.value for f, s in self.feature_sliders.items()}

        # Convert selected feature description back to feature key
        selected_feature_desc = self.sensitivity_feature_selector.value
        selected_feature = self.get_feature_key(selected_feature_desc)
        range_info = self.feature_ranges[selected_feature]

        # Create a series of values for that feature
        x_values = np.linspace(range_info['min'], range_info['max'], 20)
        rf_probs = []
        xgb_probs = []

        print("Calculating probabilities for sensitivity analysis...")
        # Evaluate model probabilities across the range of values
        for x_val in x_values:
            temp_values = current_values.copy()
            temp_values[selected_feature] = x_val

            feature_order = ["Q", "Q_1m", "WaterTemp", "DO", "pH", "SpecCond.", "OrgC", "OrgN", "PO4"]
            temp_df = pd.DataFrame([temp_values])[feature_order]

            rf_prob = self.random_forest_model.predict_proba(temp_df)[0][1]
            xgb_prob = self.xgboost_model.predict_proba(temp_df)[0][1]

            rf_probs.append(rf_prob)
            xgb_probs.append(xgb_prob)

        print("Creating sensitivity plot...")
        # Create DataFrame for plotting
        sensitivity_df = pd.DataFrame({
            'Value': x_values,
            'Random Forest': rf_probs,
            'XGBoost': xgb_probs
        })

        # Plot the sensitivity lines
        sensitivity_plot = sensitivity_df.hvplot.line(
            x='Value',
            y=['Random Forest', 'XGBoost'],
            title=f'Sensitivity Analysis: {selected_feature_desc}',
            width=600,
            height=400,
            xlabel=selected_feature_desc,
            ylabel='Probability of High Toxin Production (>4000)'
        ).opts(
            legend_position='top_right',
            tools=['hover'],
            yformatter='%.2f',
            toolbar=None,
            fontsize={'title': 12, 'labels': 10, 'xticks': 10, 'yticks': 10}
        )

        self.sensitivity_analysis_plot.object = sensitivity_plot
        print("Sensitivity analysis updated successfully")

    def create_dashboard_layout(self) -> pn.Row:
        """
        Construct the Panel layout for the dashboard, including input sliders,
        prediction results, model comparison, and sensitivity analysis.
        """
        print("Creating dashboard layout...")
        # Create headers
        input_header = pn.pane.Markdown("## Input Parameters", css_classes=['section-header'])
        results_header = pn.pane.Markdown("## Prediction Results", css_classes=['section-header'])
        comparison_header = pn.pane.Markdown("## Model Comparison", css_classes=['section-header'])
        sensitivity_header = pn.pane.Markdown("## Sensitivity Analysis", css_classes=['section-header'])

        # Apply CSS styles
        style = """
        .section-header {
            background-color: #f0f0f0;
            padding: 10px;
            margin-bottom: 10px;
        }
        """
        pn.extension(raw_css=[style])

        # Left column for inputs
        input_col = pn.Column(
            input_header,
            *[self.feature_sliders[feat] for feat in self.feature_ranges.keys()],
            width=500
        )

        # Middle column for results and comparison
        results_col = pn.Column(
            results_header,
            self.random_forest_markdown,
            self.xgboost_markdown,
            comparison_header,
            self.model_comparison_plot,
            width=700
        )

        # Right column for sensitivity analysis
        sensitivity_col = pn.Column(
            sensitivity_header,
            self.sensitivity_feature_selector,
            self.sensitivity_analysis_plot,
            width=700
        )

        print("Layout created successfully")
        return pn.Row(
            input_col,
            pn.Column(results_col, sensitivity_col),
            sizing_mode='stretch_width'
        )

    def serve_dashboard(self):
        """
        Return the servable Panel object for external use (e.g., via `panel serve ...`).
        """
        layout = self.create_dashboard_layout()
        return layout.servable()


if __name__ == "__main__":
    print("Checking model files...")
    rf_path = r'.\Model\rf_pipeline.joblib'
    xgb_path = r'.\Model\xgb_pipeline.joblib'
    print(f"RF model exists: {os.path.exists(rf_path)}")
    print(f"XGB model exists: {os.path.exists(xgb_path)}")

    print("Creating dashboard instance...")
    dashboard_app = ToxinPredictionDashboard()
    print("Dashboard instance created")

    if dashboard_app.models_loaded:
        print("Models loaded successfully, attempting to serve...")
        try:
            # Try first port
            port = 5006
            print(f"Attempting to serve on port {port}")
            pn.serve(dashboard_app.create_dashboard_layout(), show=True, port=port)
            print(f"Dashboard is running on port {port}")
        except Exception as e:
            print(f"Error serving dashboard: {str(e)}")
    else:
        print("Dashboard cannot be started due to model loading errors.")

All imports successful
Starting to initialize extensions...


HoloViews extension loaded


Panel extension loaded
Warnings filtered
Checking model files...
RF model exists: True
XGB model exists: True
Creating dashboard instance...
Starting dashboard initialization...
Attempting to load models...
Starting to load Random Forest model...
Random Forest model loaded successfully
Starting to load XGBoost model...
XGBoost model loaded successfully
Models loaded: True
Setting up feature ranges...
Feature ranges set
Setting up feature descriptions...
Feature descriptions set
Creating feature sliders...
Creating sliders...
Creating slider for Q
Creating slider for Q_1m
Creating slider for WaterTemp
Creating slider for DO
Creating slider for pH
Creating slider for SpecCond.
Creating slider for OrgC
Creating slider for OrgN
Creating slider for PO4
All sliders created
Feature sliders created
Setting up sensitivity analysis selector...
Sensitivity analysis selector created
Creating output panes...
Output panes created
Triggering initial prediction...
Updating predictions...
Updating mark

Layout created successfully
Launching server at http://localhost:5006
Dashboard is running on port 5006


Updating predictions...
Updating markdown displays...
Creating comparison plot...
Updating sensitivity plot...
Updating sensitivity analysis...
Calculating probabilities for sensitivity analysis...
Creating sensitivity plot...
Sensitivity analysis updated successfully
Predictions updated successfully
Updating predictions...
Updating markdown displays...
Creating comparison plot...
Updating sensitivity plot...
Updating sensitivity analysis...
Calculating probabilities for sensitivity analysis...


Creating sensitivity plot...
Sensitivity analysis updated successfully
Predictions updated successfully
Updating sensitivity analysis...
Calculating probabilities for sensitivity analysis...
Creating sensitivity plot...
Sensitivity analysis updated successfully
